# Revised case normalization for Hirslanden Salem 2018

Notes:




In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Salem', '2018', 'Änderungen _SA_2018')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Salem', year='2018', sheet='Änderungen _SA_2018')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-17 14:00:52.929 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 46 cases for Hirslanden Salem 2018
2022-11-17 14:00:52.935 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                             string
tranche                           string
case_id                           object
patient_id                        object
gender                            string
age_years                          int64
bfs_code                          string
duration_of_stay                   int64
pflegetage neu                    string
pccl                               int64
pccl neu                          string
old_pd                            string
primary_diagnosis                 string
added_icds                        string
removed_icds                      string
added_chops                       string
removed_chops                     string
drg                             

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,0041602986,90C3E16484AD653A,M,88,12,41602986,M171,M171,M200,3,I43B,[N183],[],[],[]
1,0041637726,D774DF60034ACAA4,W,53,19,41637726,T842,T842,M200,3,I09C,"[K564, T811]",[],[],[]
2,0041641561,4A03EA9845D13800,M,81,9,41641561,N185,N1793,M100,3,L60C,[],[],[],[]
3,0041643625,E3CD9E9C233B926F,M,83,10,41643625,E1172,E1172,M100,3,F13C,[M8617],[M8697],[],[]
4,0041644263,80BD3429C23B3A1E,W,76,10,41644263,A4152,A418,M200,3,T60E,"[J9600, B962]",[J9699],[],[]
5,0041649868,AECB468A426E92CC,M,88,14,41649868,S7208,S7208,M200,3,I46C,"[I360, I5001, R31]",[],[],[]
6,0041656883,610A5F0682A8B204,W,86,13,41656883,J100,J100,M100,3,E77E,[J9600],[],[],[]
7,0041683635,44AB9518C38C5FD0,M,88,8,41683635,S723,S723,M200,3,I08D,"[L8914, D6833]",[],[],[]
8,0041689417,AF8052F234B54DF7,W,86,3,41689417,N328,N328,M200,3,L18B,"[F009, G309]",[F03],[],[]
9,0041695908,E459BCEBEA8AD752,W,86,7,41695908,J209,J209,M100,3,E69C,[I5013],[I5019],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

ValueError: There is no data for the hospital 'Hirslanden Salem' in 2018

In [ ]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [ ]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

In [ ]:
revisions_update

In [ ]:
update_db(revisions_update, diagnoses_update, procedures_update)